In [1]:
import pyperclip,re,pyautogui,time
import pandas as pd
from random import choice,randint
import jieba.analyse
import time

pyautogui.PAUSE = 0.102

In [2]:
urllist = pd.read_csv('llist.csv',header=None)[0].to_list()

In [3]:
diyichuangkou = (14,14)
urlxy = (382,52)

name1 = (820,298)
name2 = (1325,298)

xiayiye0 = (1262,510)
xiayiye = (1262,550)

leijipingjia = (685,95)

In [4]:
def ppaste(astr):
    pyperclip.copy(astr)
    pyautogui.hotkey('ctrl', 'v')
    
def clickk(atuple):
    pyautogui.moveTo(atuple,duration=0.17,tween=pyautogui.easeInQuad,pause=0.101)
    pyautogui.click(pause=0.101)

def find_nextpage():
    pyautogui.hotkey('ctrl','f',pause=0.15)
    pyautogui.press('enter',pause=0.15)
    pyautogui.press('esc',pause=0.15)
    clickk(diyichuangkou)

def find_name():
    pyautogui.hotkey('ctrl','u',pause=0.5)
    pyautogui.hotkey('ctrl','a',pause=0.5)
    pyautogui.hotkey('ctrl','c',pause=0.5)
    sour = pyperclip.paste()
    pyautogui.hotkey('ctrl','w',pause=0.5)
    res_sou = re.findall(r'<meta name="keywords" content=".*?"/>',sour,flags=re.DOTALL)[0][31:-3]
    return res_sou

def find_from_source(asource):
    asource = asource.replace('\r\n','')
    pinglun1 = pd.Series(re.findall(r'有内容按默.{10,10000}?（匿',asource,flags=re.DOTALL))
    pinglun1 = pinglun1.map(lambda x:x[6:])
    pinglun2 = pd.Series(re.findall(r'名）.{10,10000}?（匿',asource,flags=re.DOTALL))
    pinglun = pd.concat([pinglun1,pinglun2])        

    pinglun = pinglun.str.replace('.{5,5}解释：.*','')
    pinglun = pinglun.str.replace('.*此用户没有填写评论!','')
    pinglun = pinglun.str.replace('收货.*天后追加：','。')
    pinglun = pinglun.str.replace('超级会员|名）|.{5,5}（匿','')
    
    pinglun = pinglun.str.replace('颜色分类：.*','')
    pinglun = pinglun[pinglun.map(lambda x: False if '下一页' in x else True)]
    pinglun = pinglun[pinglun.map(lambda x: False if '查看更多的' in x else True)]
    return pinglun

In [5]:
center = (940,545)
yz1 = (815,600)
yz2 = (1107,602)
def yanzheng():
    pyautogui.click((946,599),duration=1)
    pyautogui.click(center,duration=1,pause=1)
    pyautogui.press('pagedown')
    clickk(yz1)
    pyautogui.dragTo(yz2,tween=pyautogui.easeInBack,duration=2)
    #pyautogui.press('f5')
    time.sleep(5)
    
    for m in range(5):
        pyautogui.press('pagedown')
    clickk(leijipingjia)

In [6]:
urllist = urllist[:11]

In [7]:
for url_order,url in enumerate(urllist):
    url_order = url_order + 1
    df = pd.Series([url,])
    assert 'detail.tmall' in url
    
    #更改 URL 链接
    pyautogui.click(diyichuangkou,pause=0.5)
    pyautogui.click(urlxy,pause=0.5)
    pyautogui.hotkey('ctrl','a',pause=0.5)
    ppaste(url)
    pyautogui.press('enter',pause=0.5)
    for m in range(10):  #重定位-放正-页面位置
        pyautogui.press('pageup')
   
    #从页面中提取到商品名称，filename，washto-rightname
    filename = find_name()
    filename = filename.replace('*','').replace('/','-').replace('\'','').replace('\r\n','')
    print(filename)
   
    #点击到累计评价，把搜索词更改为‘下一页’
    for m in range(5):
        pyautogui.press('pagedown')
    pyautogui.click(leijipingjia,duration=1)
    
    pyautogui.hotkey('ctrl','f')
    ppaste('下一页')
    pyautogui.press('enter')
    pyautogui.press('esc')
    pyautogui.click(diyichuangkou,duration=1)
    
    #判断需要循环的页面数
    pyautogui.hotkey('ctrl','a',pause=1)
    pyautogui.hotkey('ctrl','c',pause=1)
    zhengwen = pyperclip.paste()
    pagenum = int(re.findall('累计评价 [0-9].*',zhengwen)[0][5:-1])
    pagehash = pagenum // 20 + 5
    pagehash = 120 if pagehash > 120 else pagehash
    
    for n in range(pagehash):
        clickk(diyichuangkou)
        pyautogui.hotkey('ctrl','a',pause=0.25)
        pyautogui.hotkey('ctrl','c',pause=0.15)
        zhengwen = pyperclip.paste()
        time.sleep(0.25)
        assert ('累计评价' in zhengwen) or ('验证' in zhengwen)
    
        try:
            pinglun = find_from_source(zhengwen)
            df = pd.concat([df,pinglun])
        except:
            raise TypeError
    
        if '为什么被折叠' in zhengwen:
            break
        
        if pagenum < 20:
            break
        
        if ('访问验证' in zhengwen) or ('为保证您的正' in zhengwen):
            print('访问验证')
            yanzheng()

        find_nextpage()
        pyautogui.click(xiayiye0,pause=0)
        clickk(xiayiye)
        time.sleep(1)
        
    rizhipath = str(url_order)+'_'+filename + '.xlsx'
    pd.DataFrame([]).to_excel(rizhipath)
    excelWriter = pd.ExcelWriter(rizhipath)
    
    df.drop_duplicates(inplace=True)
    df.to_excel(excelWriter,index=False,encoding='gb18030',sheet_name='源数据',header=None)
    gjc=[]
    for x in df.map(jieba.analyse.textrank):
        for y in x:
            gjc.append(y)
    pd.Series(gjc).value_counts().to_excel(excelWriter,sheet_name='源数据词频统计')
    
    excelWriter.save()
    excelWriter.close()

【倪妮同款】BOBBI BROWN芭比波朗奢金眼影 提亮打底干湿两用持妆


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\cheng.lu\AppData\Local\Temp\jieba.cache
Loading model cost 0.565 seconds.
Prefix dict has been built succesfully.


美国Urban Decay-衰败城市进口 Born to Run 21色眼影盘 16.8g
访问验证
CharlotteTilbury CT奢彩四色眼影盘哑光大地色持久便携 正品眼影
【官方正品】MAC-魅可九色眼影盘彩妆盘九宫格大地色哑光
